In [ ]:
ls

In [ ]:
import copy
import numpy as np
import numpy.linalg as la
import time
from collections import deque
from operator import itemgetter
import sys
import queue as Q
from itertools import permutations


## Class & Function defined: 

In [ ]:
class maze():
    
#   create a maze with list;
    def __init__(self, maze="mediumMaze.txt"):
        self.cost = {}
        self.raw_list = []
        self.maze_list = []
        self.width = None
        self.height = None
        self.end = [] # ready for multiple dots
        self.start = None
        self.visited = [] 
        self.solution = [] # ready for draw;
        
        self.expandedNumber = 0
        self.nodes = []
        
        with open(maze) as f:
            for line in f:
                self.raw_list.append(line.rstrip())
        self.width = len(self.raw_list[0])
        self.height = len(self.raw_list)
        
        for line in self.raw_list:
            for char in line:
                if char == "%":
                    self.maze_list.append(0)
                elif char == ".":
                    self.maze_list.append("G")
                    self.end.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                elif char == "P":
                    self.maze_list.append("P")
                    self.start = ((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                else:
                    self.maze_list.append(1)

    def accessMaze(self, x, y):
        return self.maze_list[self.getPoint(x, y)]                
    
#   Transform the index in the raw_list to the index in the maze_list
    def getPoint(self, x, y):
        if x < self.width and y < self.height:
            return y*self.width + x
        else:
            print("Index out of bound!")
            return False
  
    def getCordinate(self, point):
        return (point%self.width, point//self.width)

    def adjacentList(self, pos):
        adjacent = []
        x, y = pos
        if self.accessMaze(x+1, y) and self.getPoint(x+1, y) not in self.visited:
            adjacent.append((x+1, y))
        if self.accessMaze(x, y+1) and self.getPoint(x, y+1) not in self.visited:
            adjacent.append((x, y+1))
        if self.accessMaze(x-1, y) and self.getPoint(x-1, y) not in self.visited:
            adjacent.append((x-1, y))
        if self.accessMaze(x, y-1) and self.getPoint(x, y-1) not in self.visited:
            adjacent.append((x, y-1))
        return adjacent
    
    def dfs(self, start=(1,1), end=(59,21)):
        self.visited = []
        self.solution = []
        self.expandedNumber = 0
        self.visited.append(self.getPoint(start[0], start[1]))
        self.solution.append(start)
        
        #print("start")
        self.dfs_helper(start)
        
        
    def dfs_helper(self, current):
        self.expandedNumber += 1
        self.solution.append(current)
        
        adjacent_nodes = self.adjacentList(current)
        
        
        if current == self.end:
            return True
        
        for node in adjacent_nodes:
            nextStep= node
            self.visited.append(self.getPoint(node[0], node[1]))
            self.solution.append((node[0], node[1]))
            if self.dfs_helper(nextStep):
                return True
            self.solution.pop()
        self.solution.pop()     
        
        return
               
    def bfs(self, start=(1,1), end=(59,21)):
        queue = []
        self.visited = []
        self.solution = []
        parent = {}
        
        queue.append(start)
        self.visited.append(self.getPoint(start[0], start[1]))
                
        while queue:
            current = queue[0]
            queue = queue[1:]
            
            if current in self.end:
                # The last step to the solution, track back
                while current != self.start:
                    self.solution.append(current)
                    current = parent[current]
                self.solution.append(current)
                return True
            
            adjacent_nodes = self.adjacentList(current)
            for node in adjacent_nodes:
                parent[node] = current
                queue.append(node)
                self.visited.append(self.getPoint(node[0], node[1]))     
            
        return False
      
    def countCost(self):
        self.cost = {}
        for g in self.end:
            for index, cell in enumerate(self.maze_list):
                x = index%self.width
                y = index//self.width
                if not cell:
                    self.cost[(x,y)] = np.inf
                else:
                    self.cost[(x,y)] = abs(x - g[0]) + abs(y - g[1])
    
    # Heuristics function for single dot maze
#     def heurisFunc(self, current):
#         delta_x = current[0] - self.end[0][0]
#         delta_y = current[1] - self.end[0][1]
#         return delta_x + delta_y
               
    def best_first_search(self, start=(1,1), end=(59,21)):      
        if not self.cost:
            self.countCost()
        self.visited = []
        self.solution = []
        self.visited.append(self.getPoint(start[0], start[1]))
        self.solution.append((start[0], start[1]))
        self.greedyHelper(start)
        
        
    # Sort the adjaceny nodes list based on Manhattan Distance
    # ** Single dots implementation **
    def greedySort(self, adjacentList):
        distance = []
        for node in adjacentList:
            distance.append(self.cost[node])
          
        # get the index in term of sorting
        sorted_idx = np.argsort(distance) 
        
        # sort the original adjacent node list, ****return a numpy array!****
        return np.array(adjacentList)[sorted_idx]               
            
        
    def greedyHelper(self, current): # Almost the same as dfs
        print("current", current)
        if current in self.end:
            print("I get the destination!")
            self.solution.append(current)
            return True
        
        adjacent_nodes = self.adjacentList(current)
        if not adjacent_nodes:
            self.solution = self.solution[:-1]
            return
        
        # Sort the adjacent node list
        sorted_adj_nodes = self.greedySort(adjacent_nodes)
            
        for node in sorted_adj_nodes:
            nextStep = tuple(node)
            self.visited.append(self.getPoint(node[0], node[1]))
            self.solution.append((node[0], node[1]))
            if self.greedyHelper(nextStep):
                return True
            
        if self.solution[-1] not in self.end:
            self.solution = self.solution[:-1]       
        
        return
        
        


# Dynamically munipulate solution list!!!!
    def adjacentList_AStar(self, pos, visited, all_paths):
        adjacent = []
        x, y = pos
        if self.accessMaze(x+1, y):
            if pos not in visited:
                adjacent.append((x+1, y))
            else:
                new_cost = len(all_paths[(x+1, y)]) + self.cost[(x+1, y)]
                if new_cost <= visited[pos]:
                    visited[pos] = new_cost
                    adjacent.append((x+1, y))
                    
        if self.accessMaze(x, y+1):
            if pos not in visited:
                adjacent.append((x, y+1))
            else:
                new_cost = len(all_paths[(x, y+1)]) + self.cost[(x, y+1)]
                if new_cost <= visited[pos]:
                    visited[pos] = new_cost
                    adjacent.append((x, y+1))
                    
        if self.accessMaze(x-1, y):
            if pos not in visited:
                adjacent.append((x-1, y))
            else:
                new_cost = len(all_paths[(x-1, y)]) + self.cost[(x-1, y)]
                if new_cost <= visited[pos]:
                    visited[pos] = new_cost
                    adjacent.append((x-1, y))
                    
        if self.accessMaze(x, y-1):
            if pos not in visited:
                adjacent.append((x, y-1))
            else:
                new_cost = len(all_paths[(x, y-1)]) + self.cost[(x, y-1)]
                if new_cost <= visited[pos]:
                    visited[pos] = new_cost
                    adjacent.append((x, y-1))
                    
        return adjacent

    
    def AStar(self, start):
        self.countCost()
        visited = {}
        queue = Q.PriorityQueue()
        queue.put((self.cost[start], start))    
        cur_path = [start]
        all_paths = {}   
        all_paths[start] = cur_path
                
        while not queue.empty():
            current = queue.get()[1]
            
            if current in self.end:
                self.solution = all_paths[current]
                return True
            
            adjacent_nodes = self.adjacentList_AStar(current, visited, all_paths)
            if not adjacent_nodes:
                continue
                
            cur_path = all_paths[current]
            for node in adjacent_nodes:
                new_path = list(cur_path)
                new_path.append(node)
                all_paths[node] = new_path
                queue.put((len(new_path), node))
            
            visited[current] = self.cost[current] + len(cur_path) - 1# mark as visited
#             queue = self.AStarSort(queue, all_paths)
        
        return False
    
    
    def AStarSort(self, queue, all_paths):
        new_cost = []
        sorted_queue = []
        for node in queue:
            new_cost.append(len(all_paths[node]) + self.cost[node])
            
        sorted_idx = np.argsort(new_cost)     
        sorted_arr = np.array(queue)[sorted_idx]     
        for node in sorted_arr:
            sorted_queue.append(tuple(node))        
        return sorted_queue
    
     

    def drawSolution(self):
        solution = self.raw_list[:]
        for pos in self.solution:
            x, y = pos
            if self.raw_list[y][x] == ".":
                solution[y] = solution[y][:x] + "G" + solution[y][x+1:]
            elif self.raw_list[y][x] == "P":
                solution[y] = solution[y][:x] + "P" + solution[y][x+1:]
            else:
                solution[y] = solution[y][:x] + "." + solution[y][x+1:]
        return solution

    

 ## Single-dot MazeRunner: 

In [20]:
m = maze("testMaze.txt")
#m = maze("tinySearch.txt")
#m = maze("smallSearch.txt")

#m.A_star_search(m.start)
#m.bfs(m.start, m.end[0])
#m.dfs(m.start, m.end[0])

m.AStar(m.start)

#m.solution
m.drawSolution()

['%%%%%%%%%%%',
 '%    %.   %',
 '%    %%%% %',
 '% P..%    %',
 '%   ..G   %',
 '%%%%%%%%%%%']

In [18]:


m = maze("bigMaze.txt")
#%timeit m.dfs(start=m.start, end=m.end[0])
# %timeit m.bfs(start=m.start)
#%timeit m.AStar(start=m.start)
#%timeit m.best_first_search(start=m.start, end=m.end[0])
%timeit m.AStar(m.start)



m.drawSolution()

10 loops, best of 3: 34.2 ms per loop


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '% %           %                       %   %     %       %   %               %..G%',
 '% % %%% %%% %%% %% %%%%%%%%%% %%%%%%% %%% % %%% % % %%% % % % %%%%% %%% %%%%%.% %',
 '%   %     %   % %   %       %   %   %   % % %   % % %     % %     % % % %.....% %',
 '% %%%%    %%% % %%% % % %%% %%% % % %%% % % % %%% % %%%%% % %     % % % %.%%%%% %',
 '%         %   %   %   % % % %   % %       %     %       % % % %   % %   %.%     %',
 '%%% %% %% % %%%%% % %%% % % %  %% % %%%% %% %%% % %%%%% % % % % %%% % %%%.% %%% %',
 '%       % % %   % % % % %   %     % %       % %   %   %   %     %   % %...% % % %',
 '% %%%%%%% % % % % % % % %%% % %%%%% % %%%%%%% %%%%%%% %%%%%%% %%% %%% %.%%% % % %',
 '%     %   %   %   % % %   %   %           %     %   %   %         %   %...% % % %',
 '%%%%% % %%%%%   %%% % %%% % %%% %%%     % % %%% % % %%% % %%%%%%% %%% %%%.% % % %',
 '%         %     %   % %   % %   %   %   %   %     % 

In [5]:
m = maze("mediumMaze.txt")
#%timeit m.dfs(start=m.start, end=m.end[0])
#%timeit m.bfs(start=m.start)
#%timeit m.A_star_search(start=m.start)
%timeit m.best_first_search(start=m.start, end=m.end[0])
# m.AStar(start = m.start)
#m.bfs(start = m.start)
# m.dijsktra(m.start)
print(m.expandedNumber)



current (1, 1)
current (2, 1)
current (3, 1)
current (4, 1)
current (5, 1)
current (3, 2)
current (3, 3)
current (2, 3)
current (1, 3)
current (1, 4)
current (1, 5)
current (1, 6)
current (2, 6)
current (3, 6)
current (3, 7)
current (4, 7)
current (5, 7)
current (3, 5)
current (4, 5)
current (5, 5)
current (5, 4)
current (5, 3)
current (6, 3)
current (7, 3)
current (8, 3)
current (9, 3)
current (10, 3)
current (11, 3)
current (12, 3)
current (13, 3)
current (13, 4)
current (13, 5)
current (13, 6)
current (13, 7)
current (14, 7)
current (15, 7)
current (15, 6)
current (15, 5)
current (16, 5)
current (17, 5)
current (17, 6)
current (17, 7)
current (18, 7)
current (19, 7)
current (19, 6)
current (19, 5)
current (20, 5)
current (21, 5)
current (21, 6)
current (21, 7)
current (21, 8)
current (21, 9)
current (21, 10)
current (21, 11)
current (20, 11)
current (19, 11)
current (19, 12)
current (19, 13)
current (20, 13)
current (21, 13)
current (22, 13)
current (23, 13)
current (23, 14)
current

current (57, 18)
current (57, 19)
current (57, 20)
current (57, 21)
current (58, 21)
current (59, 21)
I get the destination!
current (1, 1)
current (2, 1)
current (3, 1)
current (4, 1)
current (5, 1)
current (3, 2)
current (3, 3)
current (2, 3)
current (1, 3)
current (1, 4)
current (1, 5)
current (1, 6)
current (2, 6)
current (3, 6)
current (3, 7)
current (4, 7)
current (5, 7)
current (3, 5)
current (4, 5)
current (5, 5)
current (5, 4)
current (5, 3)
current (6, 3)
current (7, 3)
current (8, 3)
current (9, 3)
current (10, 3)
current (11, 3)
current (12, 3)
current (13, 3)
current (13, 4)
current (13, 5)
current (13, 6)
current (13, 7)
current (14, 7)
current (15, 7)
current (15, 6)
current (15, 5)
current (16, 5)
current (17, 5)
current (17, 6)
current (17, 7)
current (18, 7)
current (19, 7)
current (19, 6)
current (19, 5)
current (20, 5)
current (21, 5)
current (21, 6)
current (21, 7)
current (21, 8)
current (21, 9)
current (21, 10)
current (21, 11)
current (20, 11)
current (19, 11)
c

current (3, 6)
current (3, 7)
current (4, 7)
current (5, 7)
current (3, 5)
current (4, 5)
current (5, 5)
current (5, 4)
current (5, 3)
current (6, 3)
current (7, 3)
current (8, 3)
current (9, 3)
current (10, 3)
current (11, 3)
current (12, 3)
current (13, 3)
current (13, 4)
current (13, 5)
current (13, 6)
current (13, 7)
current (14, 7)
current (15, 7)
current (15, 6)
current (15, 5)
current (16, 5)
current (17, 5)
current (17, 6)
current (17, 7)
current (18, 7)
current (19, 7)
current (19, 6)
current (19, 5)
current (20, 5)
current (21, 5)
current (21, 6)
current (21, 7)
current (21, 8)
current (21, 9)
current (21, 10)
current (21, 11)
current (20, 11)
current (19, 11)
current (19, 12)
current (19, 13)
current (20, 13)
current (21, 13)
current (22, 13)
current (23, 13)
current (23, 14)
current (23, 15)
current (24, 15)
current (25, 15)
current (26, 15)
current (27, 15)
current (28, 15)
current (29, 15)
current (30, 15)
current (31, 15)
current (31, 14)
current (31, 13)
current (32, 13

current (26, 15)
current (27, 15)
current (28, 15)
current (29, 15)
current (30, 15)
current (31, 15)
current (31, 14)
current (31, 13)
current (32, 13)
current (33, 13)
current (34, 13)
current (35, 13)
current (36, 13)
current (37, 13)
current (37, 14)
current (37, 15)
current (38, 15)
current (39, 15)
current (39, 16)
current (39, 17)
current (40, 17)
current (41, 17)
current (41, 16)
current (41, 15)
current (42, 15)
current (43, 15)
current (44, 15)
current (45, 15)
current (45, 16)
current (45, 17)
current (45, 18)
current (45, 19)
current (46, 19)
current (47, 19)
current (47, 18)
current (47, 17)
current (47, 16)
current (47, 15)
current (48, 15)
current (49, 15)
current (49, 14)
current (49, 13)
current (50, 13)
current (51, 13)
current (52, 13)
current (53, 13)
current (53, 14)
current (53, 15)
current (54, 15)
current (55, 15)
current (55, 16)
current (55, 17)
current (56, 17)
current (57, 17)
current (57, 18)
current (57, 19)
current (57, 20)
current (57, 21)
current (58, 2

current (19, 11)
current (19, 12)
current (19, 13)
current (20, 13)
current (21, 13)
current (22, 13)
current (23, 13)
current (23, 14)
current (23, 15)
current (24, 15)
current (25, 15)
current (26, 15)
current (27, 15)
current (28, 15)
current (29, 15)
current (30, 15)
current (31, 15)
current (31, 14)
current (31, 13)
current (32, 13)
current (33, 13)
current (34, 13)
current (35, 13)
current (36, 13)
current (37, 13)
current (37, 14)
current (37, 15)
current (38, 15)
current (39, 15)
current (39, 16)
current (39, 17)
current (40, 17)
current (41, 17)
current (41, 16)
current (41, 15)
current (42, 15)
current (43, 15)
current (44, 15)
current (45, 15)
current (45, 16)
current (45, 17)
current (45, 18)
current (45, 19)
current (46, 19)
current (47, 19)
current (47, 18)
current (47, 17)
current (47, 16)
current (47, 15)
current (48, 15)
current (49, 15)
current (49, 14)
current (49, 13)
current (50, 13)
current (51, 13)
current (52, 13)
current (53, 13)
current (53, 14)
current (53, 1

In [293]:
m = maze("mediumMaze.txt")
m.bfs(start = m.start)
m.drawSolution()

['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%P..  %       %             %         %     %           %   %',
 '%%%.%%% % %%%%% %%%%% %%%%% %%% %%%%%%% % %%% % %%%%%%% % %%%',
 '%...%.........%   % %   % %   %         %   % %     % % %   %',
 '%.%%%.%%% %%%.%%% % %%% % %%% % %%%%%% %%%% % %%%%% % % %%% %',
 '%.%...% %   %.%   %   % %                 % %   % % %       %',
 '%...%%% % % %.% % % % % %%% % %%%%%%% %%% % %%% % % %%%%%%% %',
 '% %   %   % %.  %   % %   % %     %   %   %     % %   %   % %',
 '% %%%%%%% % %.%%% %%% %%% % %%% %%% % % %%%%%%%%% %%% % % % %',
 '%   %   % %  .    % % %   %   % %   % % %     %     %   % % %',
 '% % %%% % %%%.%%%%% % % %%%%% % % %%% % % %%% %%%%% %%%%% %%%',
 '% %   % %   %.......  % %   % %     %   % % %  .....    %   %',
 '% %%%%% %%%%% %%%%%.%%% % % % %%%%%%%%% % % %%%.%%%.%%% %%% %',
 '%     %       %    .....% % %  .......%   % ....%  ...%   % %',
 '% %%% %%% %%%%% %%%%% %.%%% %%%.%%%%%.%%%%%%.%%%% %%%.%%% % %',
 '%   %   

In [31]:
m = maze("openMaze.txt")
#m.dfs(start=m.start, end=m.end[0])
#%timeit m.bfs(start=m.start)
%timeit m.AStar(start=m.start)
#m.dijsktra(m.start)
#m.best_first_search(start=m.start, end=m.end[0])

m.drawSolution()
#len(m.solution)

10 loops, best of 3: 28.1 ms per loop


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%                     %P.......     %',
 '%                     %       .     %',
 '%                     %       .     %',
 '%                     %       .     %',
 '%                     %       .     %',
 '%                     %%%%%%%%.     %',
 '%                            %.     %',
 '%                            %.     %',
 '%                            %.     %',
 '%                            %.     %',
 '%     .......................%.     %',
 '%     .%%%%%%%%%%%%%%%%%    ...     %',
 '%     .%                            %',
 '%     .%                            %',
 '%     .%                            %',
 '%     .%                            %',
 '%     .%%%%                         %',
 '%     ...G%                         %',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%']

## Multi-dot MazeRunner:

In [14]:
def bfs(self, start):
        self.bfsHelper(start)
       
def bfsHelper(self,start):
    self.start = start
    self.visited = []
    queue = []
    parent = {}
    queue.append(start)
    self.visited.append(self.getPoint(start[0], start[1]))
    while queue and len(self.end) != 0:
        current = queue[0]
        queue = queue[1:]
            
        if current in self.end:
            #print("I get to the destination!")
            self.end.remove(current)
            # The last step to the solution, track back
            self.solution.append(current)

            while current != self.start:
                self.solution.append(current)
                current = parent[current]
            self.solution = self.solution[::-1]
                
                #new search:              
            return self.bfsHelper(current)
                
        adjacent_nodes = self.adjacentList(current)
        for node in adjacent_nodes:
            parent[node] = current
            queue.append(node)
            self.visited.append(self.getPoint(node[0], node[1]))     
        
    return True

In [15]:
#m = maze("tinySearch.txt")
m = maze("smallSearch.txt")
#m = maze("mediumSearch.txt")
%timeit m.bfs(start = m.start)
m.drawSolution()

1000 loops, best of 3: 1.74 ms per loop


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%.     P.G     %      .  .   %',
 '%   %%%%%.%%%%%% %  % % %%   %',
 '%   %  % ..  %   %  % %  %  .%',
 '%     .%  .  .   %.   %  %%%%%',
 '%%%%% %%%%.%%%  %%%%%%%      %',
 '%.        ..     .    % %%%  %',
 '%% %%%%%%%%.%%%%%%%%%%% %.   %',
 '%       %  .............% %%%%',
 '%       %%%%%     %.   .     %',
 '%.% %%%    %  %   %% %%.%%%%%%',
 '%          % .%        G     %',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%']

In [96]:
# m = maze("smallSearch.txt")
# m.bfs(start = m.start)
# m.drawSolution()

## Walking through the solution path...

In [97]:
#m = maze("tinySearch.txt")
#m = maze("smallSearch.txt")
#m = maze("mediumSearch.txt")
#m = maze("openMaze.txt")
m = maze("bigMaze.txt")

m.bfs(start = m.start)
#m.dfs(start = m.start, end = m.end[0])

In [98]:
index = 0
solution = copy.deepcopy(m.raw_list)  

def walking(maze):
    for pos in maze.solution[0:index]:
        x, y = pos
        if maze.raw_list[y][x] == ".":
            solution[y] = solution[y][:x] + "G" + solution[y][x+1:]
        elif maze.raw_list[y][x] == "P":
            solution[y] = solution[y][:x] + "P" + solution[y][x+1:]
        else:
            solution[y] = solution[y][:x] + "." + solution[y][x+1:]
            
    for i in range(len(solution)):
        print(solution[i])

In [24]:
walking(m)    
index = index + 1

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% %           %                       %   %     %       %   %               %  .%
% % %%% %%% %%% %% %%%%%%%%%% %%%%%%% %%% % %%% % % %%% % % % %%%%% %%% %%%%% % %
%   %     %   % %   %       %   %   %   % % %   % % %     % %     % % % %     % %
% %%%%    %%% % %%% % % %%% %%% % % %%% % % % %%% % %%%%% % %     % % % % %%%%% %
%         %   %   %   % % % %   % %       %     %       % % % %   % %   % %     %
%%% %% %% % %%%%% % %%% % % %  %% % %%%% %% %%% % %%%%% % % % % %%% % %%% % %%% %
%       % % %   % % % % %   %     % %       % %   %   %   %     %   % %   % % % %
% %%%%%%% % % % % % % % %%% % %%%%% % %%%%%%% %%%%%%% %%%%%%% %%% %%% % %%% % % %
%     %   %   %   % % %   %   %           %     %   %   %         %   %   % % % %
%%%%% % %%%%%   %%% % %%% % %%% %%%     % % %%% % % %%% % %%%%%%% %%% %%% % % % %
%         %     %   % %   % %   %   %   %   %     %     %   %   % %     % % %   %
% %%%%%%% % %%%%

## Complexity and cost:

Big Maze:

| Search Method |  Path Cost  | Time Complexity (per loop) |
|---------------|-------------|-----------------|
|  DFS          |    236      |    39.9 ms      |
|  BFS          |    157      |    35.8 ms      |
| Greedy        |    256      |    82.7 ms      |
|   A\*         |  |  |

Medium Maze:

| Search Method |  Path Cost  | Time Complexity (per loop) |
|---------------|-------------|-----------------|
|  DFS          |           |          |
|  BFS          |           |            |
| Greedy        |           |          |
|   A\*         |  |  |

Open Maze:

| Search Method |  Path Cost  | Time Complexity (per loop) |
|---------------|-------------|-----------------|
|  DFS          |           |          |
|  BFS          |           |       |
| Greedy        |           |        |
|   A\*         |  |  |

In [332]:
class maze():
    
#   create a maze with list;
    def __init__(self, maze="mediumMaze.txt"):
        self.cost = {}
        self.raw_list = []
        self.maze_list = []
        self.nodes = []
        self.width = None
        self.height = None
        self.end = []
        self.start = None
        self.visited = [] 
        self.solution = [] # ready for draw;
        self.expandedNumber = 0
        
        with open(maze) as f:
            for line in f:
                self.raw_list.append(line.rstrip())
        self.width = len(self.raw_list[0])
        self.height = len(self.raw_list)
        
        for line in self.raw_list:
            for char in line:
                if char == "%":
                    self.maze_list.append(0)
                elif char == ".":
#                     self.maze_list.append("G")
                    self.maze_list.append(1)
                    self.end.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                    self.nodes.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                elif char == "P":
#                     self.maze_list.append("P")
                    self.maze_list.append(1)
                    self.start = ((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                    self.nodes.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                else:
                    self.maze_list.append(1)
                    self.nodes.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))

    def accessMaze(self, x, y):
        return self.maze_list[self.getPoint(x, y)]                
    
#   Transform the index in the raw_list to the index in the maze_list
    def getPoint(self, x, y):
        if x < self.width and y < self.height:
            return y*self.width + x
        else:
            print("Index out of bound!")
            return False
        
    def getCordinate(self, point):
        return (point%self.width, point//self.width)
    
    
#   Return the adjacant node who is unvisited and valid
    def adjacentList(self, pos):
        adjacent = []
        x, y = pos
        if self.accessMaze(x+1, y) and self.getPoint(x+1, y) not in self.visited:
            adjacent.append((x+1, y))
        if self.accessMaze(x, y+1) and self.getPoint(x, y+1) not in self.visited:
            adjacent.append((x, y+1))
        if self.accessMaze(x-1, y) and self.getPoint(x-1, y) not in self.visited:
            adjacent.append((x-1, y))
        if self.accessMaze(x, y-1) and self.getPoint(x, y-1) not in self.visited:
            adjacent.append((x, y-1))
        return adjacent
    
    def dfs(self, start=(1,1), end=(59,21)):
        self.visited = []
        self.solution = []
        self.expandedNumber = 0
        self.visited.append(self.getPoint(start[0], start[1]))
        self.solution.append(start)
        #print("start")
        self.dfs_helper(start)
        
        
    def dfs_helper(self, current):
        self.expandedNumber += 1
        self.solution.append(current)
        
        adjacent_nodes = self.adjacentList(current)
        
        
        if current == self.end:
            return True
        
        for node in adjacent_nodes:
            nextStep= node
            self.visited.append(self.getPoint(node[0], node[1]))
            self.solution.append((node[0], node[1]))
            if self.dfs_helper(nextStep):
                return True
            self.solution.pop()
        self.solution.pop()     
        
        return
            
        
    def bfs(self, start):
        self.expandedNumber = 0
        self.bfsHelper(start)
       
    def bfsHelper(self,start):
        self.start = start
        self.visited = []
        
        queue = []
        parent = {}
        queue.append(start)
        self.visited.append(self.getPoint(start[0], start[1]))
        self.expandedNumber += 1
        while queue:
            current = queue[0]
            queue = queue[1:]
            
            if current == self.end:
                #print("I get to the destination!")
                # The last step to the solution, track back
                self.solution.append(current)

                while current != self.start:
                    self.solution.append(current)
                    current = parent[current]
                self.solution = self.solution[::-1]
                
                return True
                
            adjacent_nodes = self.adjacentList(current)
            for node in adjacent_nodes:
                self.expandedNumber += 1
                parent[node] = current
                queue.append(node)
                self.visited.append(self.getPoint(node[0], node[1]))     
        
        return True
    
      
    def countCost(self):
        self.cost = {}
        for index, cell in enumerate(self.maze_list):
            x = index%self.width
            y = index//self.width
            if not cell:
                self.cost[(x,y)] = np.inf
            else:
                self.cost[(x,y)] = abs(x - self.end[0]) + abs(y - self.end[1])
    
               
    def best_first_search(self, start=(1,1), end=(59,21)):
        if not self.cost:
            self.countCost()
        self.visited = []
        self.solution = []
        self.expandedNumber = 0
        self.visited.append(self.getPoint(start[0], start[1]))
        self.solution.append((start[0], start[1]))
        self.greedyHelper(start)
        
    # Sort the adjaceny nodes list based on Manhattan Distance
    # ** Single dots implementation **
    def greedySort(self, adjacentList):
        distance = []
        for node in adjacentList:
            distance.append(self.cost[node])
          
        # get the index in term of sorting
        sorted_idx = np.argsort(distance) 
        
        # sort the original adjacent node list, ****return a numpy array!****
        return np.array(adjacentList)[sorted_idx]               
            
        
    def greedyHelper(self, current): # Almost the same as dfs
        self.expandedNumber += 1
        print(current)
        if current == self.end:
            print("I get the destination!")
            self.solution.append(current)
            return True
        
        adjacent_nodes = self.adjacentList(current)
        if not adjacent_nodes:
            self.solution = self.solution[:-1]
            return
        
        # Sort the adjacent node list
        sorted_adj_nodes = self.greedySort(adjacent_nodes)
            
        for node in sorted_adj_nodes:
            nextStep = tuple(node)
            self.visited.append(self.getPoint(node[0], node[1]))
            self.solution.append((node[0], node[1]))
            if self.greedyHelper(nextStep):
                return True
            
        if self.solution[-1] not in self.end:
            self.solution = self.solution[:-1]       
        
        return
        
        

    def manhattanDistance(self, cur, des): return abs(cur[0] - des[0]) + abs(cur[1] - des[1])

    def collect_all_mst(self):
        all_nodes = list(self.end)
        all_nodes.append(self.start)
        dict_path_dist = {}
        for n in all_nodes:
            shortestPathTree, disToStart = self.dijsktra(n)
            dict_path_dist[n] = [shortestPathTree, disToStart]
        return dict_path_dist
    
    
    def create_graph(self):
        self.g = graph(self.collect_all_mst())
        return self.g
    
    def calculate_cost(self):
        g = self.create_graph()
#         start = g.nodes[0]
        start = (4, 4)
        nodes = g.nodes[:]
        nodes.remove((4, 4))
        print(nodes)
        edges = g.edges
        cost = sys.maxsize
        path = []
        for node in nodes:
            edges = copy.deepcopy(g.edges)
            visited = []
#             print(node)
            new_cost = 0
            new_path = []
            new_path.append(node)
            temp = None
#             for n in nodes:
            temp_edges = None
#                 if n != node and n not in visited:
#                     print(edges[(n, node)])
            temp_edges = g.get_edge_by_last(edges, node)
#                     print(temp_edges)
            temp_edges = sorted(temp_edges, key=lambda x: x[1])
#                     print(temp_edges)
            temp = temp_edges[0][0][0]
#             print(temp)
#             visited.append(temp)
#             print(temp_edges)
            print(len(edges))
#             print(temp_edges[0][1])
            new_cost += temp_edges[0][1]
            new_path.append(temp)
#             del edges[(temp, node)]
#             del edges[(node, temp)]
            edges = g.del_edge_by_last(edges, node)
            edges = g.del_edge_by_first(edges, node)
#             edges = g.del
#             print(len(edges))
#             visited = [node, temp]
#             visited
                        
            for _ in range(len(nodes) - 2):
#                 for n in nodes:
                temp_edges = None
                if temp != node:
#                     print(edges[(n, node)])
                    temp_edges = g.get_edge_by_last(edges, temp)
                    temp_edges = sorted(temp_edges, key=lambda x: x[1])
                    temp_n = temp_edges[0][0][0]
                    visited.append(temp_n)
                    new_cost += temp_edges[0][1]
                    new_path.append(temp_n)
#                     print(len(edges))
#                     print(temp_n)
#                     del edges[(temp, temp_n)]
#                     del edges[(temp_n, temp)]
                    edges = g.del_edge_by_last(edges, temp)
                    edges = g.del_edge_by_first(edges, temp)
#                     print(len(edges))
                    print(edges)
                    temp = temp_n
                    print(temp)
#             print(visited)
            
            new_cost += edges[(start, temp)]
            new_path.append(start)
            
            
            if new_cost < cost:
                cost = new_cost
                path = new_path
        return cost, new_path
        
        
#         cost = sys.maxsize
#         order = None
#         permutation = permutations(nodes)
#         perm_cost = []
#         for p in permutation:
#             new_cost = 0
#             temp = []
#             for i in range(len(p) - 1):
#                 new_cost += edges[(p[i], p[i+1])]
#                 if new_cost >= cost:
#                     new_cost = sys.maxsize
#                     break
# #                 temp.append(edges[(p[i], p[i+1])])
#             if new_cost < cost:
#                 cost = new_cost
#                 order = p
# #             perm_cost.append(temp)
# #             return temp
#         return order
        
    
    def dijsktra_graph(self, g):
#       Record the path; key: node   value: pre;
        nodes = g.nodes[:]
        start = nodes[0]
        visited = [start]
        shortestPathTree = {start: (-1,-1)}
#       Distance from the node to the starting point; key: node   value: distance;
        disToStart = {start: 0}
        
#       Initialize the distance record; ESP, starting point has distance 0 and other nodes has maxINT;
        for node in nodes:
            if node != start:
                disToStart[node] = sys.maxsize       
#       Travel all the nodes, label their distance and pre;
        while nodes:
            min_node = None
#           Find the visited node with least distance;
            for node in nodes:
                if node in shortestPathTree:
                    if min_node == None:
                        min_node = node
                    elif disToStart[node] < disToStart[min_node]:
                        min_node = node
#           All the nodes have already been checked;
            if min_node == None:
                break
            
            nodes.remove(min_node)
#           Current cost from start to the node;
            minToStart = disToStart[min_node]
            
            adjacent_nodes = g.adjancent[min_node]
#           Label all the adjacent nodes;
            for adj_node in adjacent_nodes:
#               The edge weight is only 1; Calculate adj_node to start;
                newToStart = minToStart + g.edges[(min_node, adj_node)]
#                 newToStart = minToStart + self.manhattanDistance(min_node, self.end)
#               If the node has not been visited or the distance should be updated;
                if adj_node not in shortestPathTree or newToStart < disToStart[adj_node]:
                    shortestPathTree[adj_node] = min_node
                    disToStart[adj_node] = newToStart
        return shortestPathTree
    
#     def brute_force(self):
#         permutation = permutations(self.end)
# #         print(self.end)
# #         for i in perm:
# #             permutation.append()
#         dict_path_dist = self.collect_all_mst()
#         cost = 0
#         total_cost = []
#         for i in permutation:
#             i = list(i)
#             i.insert(0, self.start)
#             print(len(i))
#             for j in range(len(self.end)):
#                 cost += dict_path_dist[i[j]][1][i[j+1]]
#             total_cost.append(cost)
#             cost = 0
#         return total_cost
    
#   Create MST; Find the shortest path;
    def dijsktra(self, start):
#       Record the path; key: node   value: pre;
        shortestPathTree = {start: (-1,-1)}
#       Distance from the node to the starting point; key: node   value: distance;
        disToStart = {start:0}
        
#       Initialize the distance record; ESP, starting point has distance 0 and other nodes has maxINT;
        for node in range(self.width*self.height):
            if self.getCordinate(node) != start and self.maze_list[node] != 0:
                disToStart[self.getCordinate(node)] = sys.maxsize
#       self.nodes saves all the nodes; Check __init__;
        nodes = self.nodes[:]
#       Travel all the nodes, label their distance and pre;
        while nodes:
            min_node = None
#           Find the visited node with least distance;
            for node in nodes:
                if node in shortestPathTree:
                    if min_node == None:
                        min_node = node
                    elif disToStart[node] < disToStart[min_node]:
                        min_node = node
#           All the nodes have already been checked;
            if min_node == None:
                break
            
            nodes.remove(min_node)
#           Current cost from start to the node;
            minToStart = disToStart[min_node]
            
            adjacent_nodes = self.adjacentList(min_node)
#           Label all the adjacent nodes;
            for adj_node in adjacent_nodes:
#               The edge weight is only 1;
                newToStart = minToStart + 1
#                 newToStart = minToStart + self.manhattanDistance(min_node, self.end)
#               If the node has not been visited or the distance should be updated;
                if adj_node not in shortestPathTree or newToStart < disToStart[adj_node]:
                    shortestPathTree[adj_node] = min_node
                    disToStart[adj_node] = newToStart
        return shortestPathTree, disToStart
                
# #       Draw the solution;
#         end = self.end
#         solution = self.raw_list[:] # faster than deep copy;  
#         while end != (-1,-1):
#             x, y = end
#             if self.raw_list[y][x] == ".":
#                 solution[y] = solution[y][:x] + "G" + solution[y][x+1:]
#             elif self.raw_list[y][x] == "P":
#                 solution[y] = solution[y][:x] + "P" + solution[y][x+1:]
#             else:
#                 solution[y] = solution[y][:x] + "." + solution[y][x+1:]
#             end = shortestPathTree[end]
#         return solution
            
                
            
            
        
        
        
                
    
     
    

    def drawSolution(self):
        solution = self.raw_list[:] # faster than deep copy;  
        for pos in self.solution:
            x, y = pos
            if self.raw_list[y][x] == ".":
                solution[y] = solution[y][:x] + "G" + solution[y][x+1:]
            elif self.raw_list[y][x] == "P":
                solution[y] = solution[y][:x] + "P" + solution[y][x+1:]
            else:
                solution[y] = solution[y][:x] + "." + solution[y][x+1:]
        return solution

In [333]:
class graph():
    
    def __init__(self, dict_path_dist):
#         self.start = dict_path_dist[0]
        self.start = (4, 4)
        self.nodes = []
        self.edges = {}
        self.adjancent = {}
        for node in dict_path_dist:
            self.insert_node(node)
            
        for x in self.nodes:
            for y in self.nodes:
                if x != y:
                    self.insert_edge(x, y, dict_path_dist[x][1][y])
        
            
    def insert_node(self, node):
        self.nodes.append(node)
        
    def insert_edge(self, x, y, weight):
        if (x, y) not in self.edges:
            self.edges[(x, y)] = weight
        if x not in self.adjancent:
            self.adjancent[x] = [y]
        else:    
            self.adjancent[x].append(y)
        
    def get_edge_by_last(self, edges, y):
        output = []
        for x in self.nodes:
            if (x, y) in edges and x != (4,4):
                output.append(((x, y) ,self.edges[x, y]) )
        return output
    
    def del_edge_by_last(self, edges, y):
        for x in self.nodes:
            if (x, y) in edges:
                del edges[(x, y)]
        return edges
    
    def del_edge_by_first(self, edges, x):
        for y in self.nodes:
            if(x, y) in edges:
                del edges[(x, y)]
        return edges
    

In [334]:
m = maze("tinySearch.txt")
m.create_graph()
# print(len(m.g.edges))
# print(len(m.g.nodes))
# m.g.adjancent[1,1]
m.calculate_cost()

[(1, 1), (8, 1), (3, 2), (6, 3), (8, 3), (2, 4), (1, 5), (4, 5), (7, 5), (8, 6), (1, 7), (6, 7)]
156
{((8, 1), (6, 3)): 6, ((8, 1), (8, 3)): 2, ((8, 1), (2, 4)): 11, ((8, 1), (1, 5)): 11, ((8, 1), (4, 5)): 8, ((8, 1), (7, 5)): 5, ((8, 1), (8, 6)): 5, ((8, 1), (1, 7)): 13, ((8, 1), (6, 7)): 8, ((8, 1), (4, 4)): 7, ((6, 3), (8, 1)): 6, ((6, 3), (8, 3)): 4, ((6, 3), (2, 4)): 7, ((6, 3), (1, 5)): 7, ((6, 3), (4, 5)): 4, ((6, 3), (7, 5)): 3, ((6, 3), (8, 6)): 5, ((6, 3), (1, 7)): 9, ((6, 3), (6, 7)): 4, ((6, 3), (4, 4)): 3, ((8, 3), (8, 1)): 2, ((8, 3), (6, 3)): 4, ((8, 3), (2, 4)): 9, ((8, 3), (1, 5)): 9, ((8, 3), (4, 5)): 6, ((8, 3), (7, 5)): 3, ((8, 3), (8, 6)): 3, ((8, 3), (1, 7)): 11, ((8, 3), (6, 7)): 6, ((8, 3), (4, 4)): 7, ((2, 4), (8, 1)): 11, ((2, 4), (6, 3)): 7, ((2, 4), (8, 3)): 9, ((2, 4), (1, 5)): 2, ((2, 4), (4, 5)): 3, ((2, 4), (7, 5)): 6, ((2, 4), (8, 6)): 8, ((2, 4), (1, 7)): 4, ((2, 4), (6, 7)): 7, ((2, 4), (4, 4)): 4, ((1, 5), (8, 1)): 11, ((1, 5), (6, 3)): 7, ((1, 5), (

(37,
 [(6, 7),
  (7, 5),
  (8, 6),
  (8, 3),
  (8, 1),
  (6, 3),
  (3, 2),
  (1, 1),
  (2, 4),
  (1, 5),
  (1, 7),
  (4, 5),
  (4, 4)])

In [335]:
m.drawSolution()

['%%%%%%%%%%',
 '%.  %   .%',
 '% %.% %% %',
 '% %   .%.%',
 '% .%P%   %',
 '%.  .  . %',
 '% %%%% %.%',
 '%.    .% %',
 '%%%%%%%%%%']

In [108]:
m.g.edges

{((1, 1), (1, 5)): 4,
 ((1, 1), (1, 7)): 6,
 ((1, 1), (2, 4)): 4,
 ((1, 1), (3, 2)): 3,
 ((1, 1), (4, 4)): 6,
 ((1, 1), (4, 5)): 7,
 ((1, 1), (6, 3)): 7,
 ((1, 1), (6, 7)): 11,
 ((1, 1), (7, 5)): 10,
 ((1, 1), (8, 1)): 11,
 ((1, 1), (8, 3)): 11,
 ((1, 1), (8, 6)): 12,
 ((1, 5), (1, 1)): 4,
 ((1, 5), (1, 7)): 2,
 ((1, 5), (2, 4)): 2,
 ((1, 5), (3, 2)): 7,
 ((1, 5), (4, 4)): 4,
 ((1, 5), (4, 5)): 3,
 ((1, 5), (6, 3)): 7,
 ((1, 5), (6, 7)): 7,
 ((1, 5), (7, 5)): 6,
 ((1, 5), (8, 1)): 11,
 ((1, 5), (8, 3)): 9,
 ((1, 5), (8, 6)): 8,
 ((1, 7), (1, 1)): 6,
 ((1, 7), (1, 5)): 2,
 ((1, 7), (2, 4)): 4,
 ((1, 7), (3, 2)): 9,
 ((1, 7), (4, 4)): 6,
 ((1, 7), (4, 5)): 5,
 ((1, 7), (6, 3)): 9,
 ((1, 7), (6, 7)): 5,
 ((1, 7), (7, 5)): 8,
 ((1, 7), (8, 1)): 13,
 ((1, 7), (8, 3)): 11,
 ((1, 7), (8, 6)): 10,
 ((2, 4), (1, 1)): 4,
 ((2, 4), (1, 5)): 2,
 ((2, 4), (1, 7)): 4,
 ((2, 4), (3, 2)): 7,
 ((2, 4), (4, 4)): 4,
 ((2, 4), (4, 5)): 3,
 ((2, 4), (6, 3)): 7,
 ((2, 4), (6, 7)): 7,
 ((2, 4), (7, 5)): 6,
 

In [177]:
m.g.nodes

[(1, 1),
 (8, 1),
 (3, 2),
 (6, 3),
 (8, 3),
 (2, 4),
 (1, 5),
 (4, 5),
 (7, 5),
 (8, 6),
 (1, 7),
 (6, 7),
 (4, 4)]

In [336]:
m.dijsktra((6,7))

({(1, 1): (2, 1),
  (1, 2): (1, 3),
  (1, 3): (1, 4),
  (1, 4): (2, 4),
  (1, 5): (2, 5),
  (1, 6): (1, 7),
  (1, 7): (2, 7),
  (2, 1): (3, 1),
  (2, 4): (2, 5),
  (2, 5): (3, 5),
  (2, 7): (3, 7),
  (3, 1): (3, 2),
  (3, 2): (3, 3),
  (3, 3): (4, 3),
  (3, 5): (4, 5),
  (3, 7): (4, 7),
  (4, 3): (5, 3),
  (4, 4): (4, 5),
  (4, 5): (5, 5),
  (4, 7): (5, 7),
  (5, 1): (5, 2),
  (5, 2): (5, 3),
  (5, 3): (6, 3),
  (5, 5): (6, 5),
  (5, 7): (6, 7),
  (6, 1): (5, 1),
  (6, 3): (6, 4),
  (6, 4): (6, 5),
  (6, 5): (6, 6),
  (6, 6): (6, 7),
  (6, 7): (-1, -1),
  (7, 1): (6, 1),
  (7, 4): (6, 4),
  (7, 5): (6, 5),
  (8, 1): (8, 2),
  (8, 2): (8, 3),
  (8, 3): (8, 4),
  (8, 4): (7, 4),
  (8, 5): (7, 5),
  (8, 6): (8, 5),
  (8, 7): (8, 6)},
 {(1, 1): 11,
  (1, 2): 10,
  (1, 3): 9,
  (1, 4): 8,
  (1, 5): 7,
  (1, 6): 6,
  (1, 7): 5,
  (2, 1): 10,
  (2, 4): 7,
  (2, 5): 6,
  (2, 7): 4,
  (3, 1): 9,
  (3, 2): 8,
  (3, 3): 7,
  (3, 5): 5,
  (3, 7): 3,
  (4, 3): 6,
  (4, 4): 5,
  (4, 5): 4,
  (4, 7):